In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import corpus_bleu

# Assuming you have your data prepared and tokenized
# X_train, X_test, y_train, y_test = ...

import numpy as np

# Assuming X_train and X_test are DataFrames
X_train_array = X_train.to_numpy()
X_test_array = X_test.to_numpy()
y_train.reset_index(drop=True, inplace=True)
y_test.reset_index(drop=True, inplace=True)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train_array).float()
y_train_tensor = torch.tensor(y_train).float()
X_test_tensor = torch.tensor(X_test_array).float()
y_test_tensor = torch.tensor(y_test).float()


# Define RNN model
class RNN(nn.Module):
    def _init_(self, input_size, hidden_size, output_size):
        super(RNN, self)._init_()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.rnn(x)
        out = self.fc(out[:, -1, :])  # Get output from the last time step
        return out

# Define evaluation function
def evaluate(model, criterion, data_loader):
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for inputs, targets in data_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item() * len(inputs)
    return total_loss / len(data_loader.dataset)

# Define hyperparameters
input_size = X_train_tensor.size(2)
output_size = y_train_tensor.size(2)
hidden_size = 64
batch_size = 32
learning_rate = 0.001
num_epochs = 10

# Create DataLoader
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Initialize RNN model, loss function, and optimizer
model_rnn = RNN(input_size, hidden_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model_rnn.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model_rnn.train()
    total_loss = 0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model_rnn(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * len(inputs)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {total_loss / len(train_loader.dataset)}')

# Evaluate the model
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, batch_size=batch_size)
test_loss = evaluate(model_rnn, criterion, test_loader)
print(f'Test Loss: {test_loss}')

# Calculate BLEU score (assuming y_true and y_pred are lists of sentences)
# bleu_score = corpus_bleu(y_true, y_pred)